In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)
import time


#### Model

In [ ]:
# adjust these constants, then run entire notebook.

model_name = "cooleys"
model_version = "a"

#### Paths

In [ ]:
from pathlib import Path

model_path = Path(f"./models/{model_name}")
checkpoint = model_path / f"{model_version}/checkpoint/"
metadata_path = Path(f"./models/metadata.yaml")

output_path = Path(f"./results/{model_name}/{model_version}/")

#### Parameters

In [ ]:
base_test_parameters = {
    'max_length': 128,
    'temperature': 0.8,
    'no_repeat_ngram_size': 10,
    'do_sample': True,
    'top_k': 50,
    'early_stopping': True,
}

test_runs = [
    {
        'max_length': 128,
        'temperature': 0.6,
    },
    {
        'max_length': 128,
        'temperature': 0.8,
    },
    {b
        'max_length': 128,
        'temperature': 1.0,
    },
    {
        'max_length': 256,
        'temperature': 0.8,
    },
    {
        'max_length': 512,
        'temperature': 0.8,
    },
    {
        'max_length': 128,
        'top_k': 50,
        'temperature': 0.6,
        'no_repeat_ngram_size': 10,
        'do_sample': False,
        'num_beams': 5
    },
     {
        'max_length': 128,
        'top_k': 50,
        'temperature': 1.0,
        'no_repeat_ngram_size': 10,
        'do_sample': False,
        'num_beams': 5
    },
]

test_parameters = []
for i in range(0, len(test_runs)):
    run_parameters = base_test_parameters.copy()
    run_parameters.update(test_runs[i])
    test_parameters.append(run_parameters)

#### Prompts

In [182]:
two_prompts= ["""R: reel
M: 4/4
K: Edorian
|: E2 B E d E B E |""",
             
      """R: jig
M: 6/8
K: Gmajor
|: G3 G A B |"""]


test_prompts = [
    
    # drowsy maggie - Reel, Edor
    """R: reel
M: 4/4
K: Edorian
|: E2 B E d E B E |""",
    
    # kesh jig - Jig, Gmaj
    """R: jig
M: 6/8
K: Gmajor
|: G3 G A B |""",
    
    # john ryan's - Polka, Dmaj 
    """R: polka
M: 2/4
K: Dmajor
d d B/ c/ d/ B/ |""",
    
    # king of the fairies - Hornpipe
    """R: hornpipe
M: 4/4
K: Edorian
|: B,2 | E D E F G F G A""",
    
    # inisheer - Waltz
    """R: waltz
M: 3/4
K: Gmajor
B2 B A B d |""",
    
    # the butterfly - Slip Jig
    """R: slip jig
M: 9/8
K: Eminor
|: B2 E G2 E F3 |""",
    
    # banish misfortune - Dmix
    """R: jig
M: 6/8
K: Dmixolydian
f e d c A G |""",
    
    # tam lin - Dmin, low register
    """R: reel
M: 4/4
K: Dminor
A,2 D A, F A, D A, |""",
    
    # cliffs of moher - Ador, high register
    """R: jig
M: 6/8
K: Adorian
|: a3 b a g |""",
    
    # silver spear - no closing barline
    """R: reel
M: 4/4
K: Dmajor
A |: F A (3 A A A B A F A""",
    
    # unspecified tune types and meters
    "R: jig",
    "R: reel",
    "R: polka",
    "R: waltz",
    "R: hornpipe",
    "R: slip jig",
    "M: 4/4",
    "M: 6/8",
    "M: 3/4",
    "M: 2/4",
    "M: 9/8",
    
    # single notes
    "G",
    "e"
]

#### HTML Config

In [ ]:
# for compiling html file
html_header = """<html>
<meta charset="UTF-8"/>
<script src="http://moinejf.free.fr/js/abcweb-1.js"></script>
<script src="http://moinejf.free.fr/js/snd-1.js"></script>
<style>svg {display:block}</style>
<title></title>
</head>
<body bgcolor="#faf0e6">"""

html_footer = """</body>
</html>"""

html_abc_prefix = """<script type="text/vnd.abc">
%abc-2.2
%%pagewidth 14cm
%%bgcolor white
%%topspace 0
%%composerspace 0
%%leftmargin 0.8cm
%%rightmargin 0.8cm

"""

html_abc_postfix = """</script>\n\n"""

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, local_files_only=True)

#### Model Metadata

In [ ]:
import yaml, torch

with open(metadata_path, "r") as file:
    m = yaml.safe_load(file)
    general_metadata = {
        'model_name': model_name,
        'model_description': m['models'][model_name]['description'],
        'dataset': m['models'][model_name]['dataset'],
        'version': model_version,
        'version_description': m['models'][model_name]['versions'][model_version]['description']
    }


with open(checkpoint / 'trainer_state.json', "r") as file:
    m = json.load(file)
    training_metadata = {
        k: v for k,v in m.items() if k in ['epoch', 'global_step']
    }

model_metadata_fields = [
    "model_type",
    "vocab_size",
    "n_ctx",
    "n_embd",
    "n_head",
    "n_inner",
    "n_layer",
    "n_positions",
]

model_metadata = {
    k: getattr(model.config, k) for k in model_metadata_fields
}

#### File Generation

In [ ]:
def generate(start_text = "R: reel", parameters = test_parameters):
    # encoding the input text
    if model_name in ['cooleys']:
        input_ids = tokenizer.encode(start_text, return_tensors='pt')[:, :-1]
    else:
        input_ids = tokenizer.encode(start_text, return_tensors='pt')
    output = model.generate(input_ids, **parameters)
    return output

In [183]:
import os

def test_suite(
    path = model_path,
    test_parameters = test_parameters,
    prompts = test_prompts,
):
    
    current_time = time.strftime("%Y-%m-%dT%H:%M:%S", time.localtime())
    output_dir = output_path / current_time
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    txt_file_name = output_dir / "results.txt"
    html_file_name = output_dir / "results.html"
    print(f"running tests on model at {path}")
    print(f"saving to {txt_file_name} and {html_file_name}")
    
    with open(txt_file_name, 'a') as txt_file, open(html_file_name, 'a') as html_file:
        
        html_file.write(html_header)
        
        txt_file.write(f"{current_time}\n\n")
        html_file.write(f"<p>{current_time}</p>\n")
        
        txt_file.write(f"path_to_model: {path}\n\n")
        html_file.write(f"<p>path_to_model: {path}</p>\n")
        
        # metadata
        txt_file.write(f"general_metadata:\n")
        html_file.write(f"<p>general_metadata:\n<ul>\n")
        for k in general_metadata:
            txt_file.write(f"    {k}: {general_metadata[k]}\n")
            html_file.write(f"<li>{k}: {general_metadata[k]}\n</li>\n")
        txt_file.write("\n=========\n\n")
        html_file.write("</ul>\n</p>\n")
        
        # model metadata
        txt_file.write(f"model_metadata:\n")
        html_file.write(f"<p>model_metadata:\n<ul>\n")
        for k in model_metadata:
            txt_file.write(f"    {k}: {model_metadata[k]}\n")
            html_file.write(f"<li>{k}: {model_metadata[k]}\n</li>\n")
        txt_file.write("\n=========\n")
        html_file.write("</ul>\n</p>\n")
        
        # training metadata
        txt_file.write(f"training_metadata:\n")
        html_file.write(f"<p>training_metadata:\n<ul>\n")
        for k in training_metadata:
            txt_file.write(f"    {k}: {training_metadata[k]}\n")
            html_file.write(f"<li>{k}: {training_metadata[k]}\n</li>\n")
        txt_file.write("\n=========\n")
        html_file.write("</ul>\n</p>\n")
        
        # prompt index
        html_file.write(f"<p>prompts:\n<ul>\n")
        for i, prompt in enumerate(prompts):
            html_file.write('<li><a href="#prompt{num}">Prompt {num} - {p}</a></li>\n'.format(num = i + 1,
                                                                                              p = prompt))
        html_file.write("</ul>\n</p>\n")
        
        prompt_count = 0
        test_count = 0
        for prompt in prompts:
            if model_name not in ['ashplant']:
                prompt = prompt.replace('\n', '\t')
            prompt_count += 1
            print("running prompt {}/{}: {}".format(prompt_count, len(prompts), prompt))
            
            prompt_counter_text = "prompt {}:\n".format(prompt_count)
            prompt_text = "\n{}\n\n\n".format(prompt)
            txt_file.write(prompt_counter_text)
            txt_file.write(prompt_text)
            html_file.write('<h3 id="prompt{c}">Prompt {c} - {p}</h3>\n'.format(c = prompt_count,
                                                                 p = prompt))
            html_file.write("<p>")
            
            
            for test in test_parameters:
                
                output = generate(prompt, test)[0]
    
                s = tokenizer.decode(output)
                if model_name in ['cooleys']:
                    special_tokens=[
                        "<s>",
                        "</s>",
                        "<pad>",
                        "<unk>",
                        "<mask>",
                    ]
                    for t in special_tokens:
                        s = s.replace(t, '')
                if model_name not in ['ashplant']:
                    s = s.replace('\t', '\n')
                
                        # training metadata
                txt_file.write(f"test_parameters:\n")
                html_file.write(f"<p>test_parameters:\n<ul>\n")
                for k in test:
                    txt_file.write(f"    {k}: {test[k]}\n")
                    html_file.write(f"<li>{k}: {test[k]}\n</li>\n")
                txt_file.write("\n=========\n")
                html_file.write("</ul>\n</p>\n")
        
                txt_file.write(f"model_output:\n")
                html_file.write(f"model_output:\n")
                txt_file.write('X: {}\n'.format(test_count))
                txt_file.write(s + "\n---\n")
                html_file.write("\n<pre>X: {}\n".format(test_count))
                html_file.write(s + "</pre>\n\n")
                html_file.write(html_abc_prefix)
                html_file.write("X: {}\n".format(test_count))
                html_file.write(s.replace(" ", "") + "\n\n")
                html_file.write(html_abc_postfix)
                
                test_count += 1
            
            html_file.write("</p>")
            txt_file.write("\n---------\n\n")
            
        
        html_file.write(html_footer)
        print("all prompts completed!")

In [184]:
test_suite()

running tests on model at models/cooleys
saving to results/cooleys/b/2021-07-30T00:55:49/results.txt and results/cooleys/b/2021-07-30T00:55:49/results.html
running prompt 1/23: R: reel	M: 4/4	K: Edorian	|: E2 B E d E B E |


KeyboardInterrupt: 

### Run All

In [ ]:
# This is very sloppy but this is a loop to run the test code for all of the existing models when we want to
# TODO: Re-org the code so it's not duplicated 

In [185]:
with open(metadata_path, "r") as file:
    m = yaml.safe_load(file)

model_list = [
    (model, version) 
    for model in m['models'] 
    for version in m['models'][model]['versions']
]

In [ ]:
import yaml, torch

for model_name, model_version in model_list:
    
    model_path = Path(f"./models/{model_name}")
    checkpoint = model_path / f"{model_version}/checkpoint/"
    metadata_path = Path(f"./models/metadata.yaml")
    output_path = Path(f"./results/{model_name}/{model_version}/")
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, local_files_only=True)
    model = AutoModelForCausalLM.from_pretrained(checkpoint, local_files_only=True)

    with open(metadata_path, "r") as file:
        m = yaml.safe_load(file)
        general_metadata = {
            'model_name': model_name,
            'model_description': m['models'][model_name]['description'],
            'dataset': m['models'][model_name]['dataset'],
            'version': model_version,
            'version_description': m['models'][model_name]['versions'][model_version]['description']
        }


    with open(checkpoint / 'trainer_state.json', "r") as file:
        m = json.load(file)
        training_metadata = {
            k: v for k,v in m.items() if k in ['epoch', 'global_step']
        }

    model_metadata_fields = [
        "model_type",
        "vocab_size",
        "n_ctx",
        "n_embd",
        "n_head",
        "n_inner",
        "n_layer",
        "n_positions",
    ]

    model_metadata = {
        k: getattr(model.config, k) for k in model_metadata_fields
    }
    test_suite()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


running tests on model at models/cooleys
saving to results/ashplant/a/2021-07-30T00:56:24/results.txt and results/ashplant/a/2021-07-30T00:56:24/results.html
running prompt 1/23: R: reel
M: 4/4
K: Edorian
|: E2 B E d E B E |


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


running prompt 2/23: R: jig
M: 6/8
K: Gmajor
|: G3 G A B |


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

running prompt 22/23: G


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


running prompt 23/23: e


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


all prompts completed!
running tests on model at models/cooleys
saving to results/cooleys/a/2021-07-30T01:18:03/results.txt and results/cooleys/a/2021-07-30T01:18:03/results.html
running prompt 1/23: R: reel	M: 4/4	K: Edorian	|: E2 B E d E B E |
running prompt 2/23: R: jig	M: 6/8	K: Gmajor	|: G3 G A B |
running prompt 3/23: R: polka	M: 2/4	K: Dmajor	d d B/ c/ d/ B/ |
running prompt 4/23: R: hornpipe	M: 4/4	K: Edorian	|: B,2 | E D E F G F G A
running prompt 5/23: R: waltz	M: 3/4	K: Gmajor	B2 B A B d |
running prompt 6/23: R: slip jig	M: 9/8	K: Eminor	|: B2 E G2 E F3 |
running prompt 7/23: R: jig	M: 6/8	K: Dmixolydian	f e d c A G |
running prompt 8/23: R: reel	M: 4/4	K: Dminor	A,2 D A, F A, D A, |
running prompt 9/23: R: jig	M: 6/8	K: Adorian	|: a3 b a g |
running prompt 10/23: R: reel	M: 4/4	K: Dmajor	A |: F A (3 A A A B A F A
running prompt 11/23: R: jig
running prompt 12/23: R: reel
running prompt 13/23: R: polka
running prompt 14/23: R: waltz
running prompt 15/23: R: hornpipe
running